In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import os
import numpy as np
from utils import remove_specular_component, apply_threshold_segmentation, find_most_informative_region, \
    extract_roi, extract_features

In [ ]:
# Function to load and resize images from a directory
def load_images_from_directory(directory, num_images=10, size=(256, 256)):
    all_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.jpg')]
    selected_files = random.sample(all_files, num_images)
    images = [cv2.imread(file) for file in selected_files]
    resized_images = [cv2.resize(img, size, interpolation=cv2.INTER_AREA) for img in images]
    return resized_images

In [ ]:
# Function to process images
def process_images(images):
    segmented_images = []
    for image in images:
        specular_free_image_safe = remove_specular_component(image)
        color_regions_image = apply_threshold_segmentation(specular_free_image_safe)
        row = find_most_informative_region(color_regions_image)
        valid_image = cv2.bitwise_and(image, image, mask=color_regions_image)
        row2 = extract_roi(valid_image, row, height=70)
        print(valid_image.shape)
        segmented_images.append(row2)
    return segmented_images

In [ ]:
# Directory where images are stored
directory_path = 'cars_images/'

# Load images
images = load_images_from_directory(directory_path)

# Process images
segmented_images = process_images(images)

# Display segmented images
fig, axes = plt.subplots(1, 10, figsize=(30, 20))  # Adjust the size as needed
for i, segmented in enumerate(segmented_images):
    axes[i].imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f'Segmented Image {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()